In [1]:
import json
import pandas as pd

In [2]:
%matplotlib inline

In [3]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 999)

In [4]:
%%time
from pymongo import MongoClient

MONGO_HOST= 'mongodb://localhost/twitterdata'
client = MongoClient(MONGO_HOST)
db = client.twitterdata
coll = db.abortolegal_agosto

tweets = []
users = {}
cols_tweet = ['id', 'created_at', 'favorite_count', 'retweet_count', 'text', 'favorited', 'retweeted']
cols_user = [
             'screen_name',
             'location',
             'followers_count',
             'statuses_count',
             'description',
             'friends_count',
             'favourites_count',
             'created_at',
             'id',
             'name',             
             'verified']

for t in coll.find():
    tr = [t[c] for c in cols_tweet] + [t['user']['name'], t['user']['id']]
    tr.append(t['retweeted_status']['id'] if t['retweeted'] else t['id'])
    tweets.append(tr)
    uid = t['user']['id'] 
    if uid not in users:
        users[uid] = [t['user'][c] for c in cols_user]

CPU times: user 23.8 s, sys: 809 ms, total: 24.6 s
Wall time: 24.5 s


In [5]:
%%time
tweets = pd.DataFrame(tweets, columns=cols_tweet + ['user_name', 'user_id', 'original_id'])

CPU times: user 240 ms, sys: 0 ns, total: 240 ms
Wall time: 239 ms


In [6]:
%%time
users = pd.DataFrame(users.values(), columns=cols_user)

CPU times: user 195 ms, sys: 110 µs, total: 195 ms
Wall time: 194 ms


## Guardar datos en formato accesible

In [7]:
tweets.head()

,id,created_at,favorite_count,retweet_count,text,favorited,retweeted,user_name,user_id,original_id
0,1025181421961392135,Fri Aug 03 00:47:52 +0000 2018,0,0,🔹 5 Mitos sobre el aborto que todos debemos saber [VÍDEO] https://t.co/QVLOyPu7tu,False,False,Revista Digital,2208347839,1025181421961392135
1,1025181422347268097,Fri Aug 03 00:47:52 +0000 2018,0,0,"RT @antotripelli: Mi mamá hablando con una amiga:\nMadre: estas a favor del aborto?\nAmiga: no, la verdad no. Una amiga se murio por abortar…",False,False,Mimi,878157175,1025181422347268097
2,1025181423639113728,Fri Aug 03 00:47:53 +0000 2018,0,0,El aborto que se perdió tu vieja jajajajajajajajajajjajajajaja,False,False,Delfina💚,984930721373573120,1025181423639113728
3,1025181426361221120,Fri Aug 03 00:47:53 +0000 2018,0,0,RT @cortalabochx: Si se legaliza el aborto los pro vida no van a votar a macri https://t.co/DNeGMRn2ZP,False,False,Mr. Drama👑,630668646,1025181426361221120
4,1025181426608730113,Fri Aug 03 00:47:53 +0000 2018,0,0,RT @AgustinLajeQuot: Ya saben que responder cuando te vienen con el mito de: “El aborto es la interrupción del embarazo”. https://t.co/vqPa…,False,False,Luigi Venegas,403723840,1025181426608730113


In [8]:
clean_text = lambda s: s.replace('\n', ' ').replace('\r', ' ')

In [11]:
tweets.text = tweets.text.apply(clean_text)

In [12]:
tweets.to_csv('../data/csvs/tweets_agosto.csv', index=False, encoding='utf8')

In [14]:
users.columns

Index([u'screen_name', u'location', u'followers_count', u'statuses_count',
       u'description', u'friends_count', u'favourites_count', u'created_at',
       u'id', u'name', u'verified'],
      dtype='object')

In [16]:
users.description = users.description.astype(unicode)

In [17]:
users.description = users.description.apply(clean_text)

In [18]:
users.head()

,screen_name,location,followers_count,statuses_count,description,friends_count,favourites_count,created_at,id,name,verified
0,abortoensenado,"Buenos Aires, Argentina",619,1028,"Sólo hay una cosa más grande que el amor a la libertad, y es el odio a quien te la quita 💚",1264,305,Wed Jun 20 17:12:43 +0000 2018,1009484201660383232,Aborto En El Senado,False
1,MotonetaZ,None,12,265,None,66,189,Thu Jun 15 20:07:02 +0000 2017,875444558745567232,Lucas,False
2,AliseValix0210,None,34,3218,"ARMY ,❤ BTS ❤, HAY QUE AMARLOS A TODOS.",92,5076,Mon Nov 20 23:30:37 +0000 2017,932753072681517058,Alise,False
3,MilenaDiser27,None,28,106,artist👩🏻‍🎨🎸🎶 🖤AHS🖤 cuenta nueva🤙🏽🔥 snap👻: milenadiser👽,114,113,Mon Jun 11 01:46:56 +0000 2018,1005989729542733827,FREAK👽,False
4,ConLaGenteRos,"Rosario, Argentina",2325,15230,Diario digital de Rosario,449,28,Sun Dec 10 21:06:45 +0000 2017,939964624711057412,Con La Gente,False


In [19]:
users.to_csv('../data/csvs/users_agosto.csv', index=False, encoding='utf8')